In [467]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
df=pd.read_csv("/content/drug200.csv")
df


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [468]:
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [469]:
df.isna().sum()

,0
Age,0
Sex,0
BP,0
Cholesterol,0
Na_to_K,0
Drug,0


In [470]:
df.dtypes

,0
Age,int64
Sex,object
BP,object
Cholesterol,object
Na_to_K,float64
Drug,object


In [471]:
#data binning
bin_age = [0,19,29,39,49,59,69,80]
category_age = ['<20s','20s','30s','40s','50s','60s','>60s']
df['Age_binned'] = pd.cut(df['Age'],bins=bin_age,labels=category_age)
df_drug = df.drop(['Age'],axis=1)

Na to K

In [472]:
bin_NatoK = [0,9,19,29,50]
category_NatoK = ['<10','10-20','20-30','>30']
df['Na_to_K_binned'] = pd.cut(df['Na_to_K'],bins=bin_NatoK,labels=category_NatoK)
df_drug = df.drop(['Na_to_K'],axis=1)

In [473]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [474]:
X = df.drop(["Drug"],axis=1)
y = df["Drug"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3 ,random_state = 0)

feature engineering

In [475]:
from sklearn.preprocessing import OneHotEncoder


In [476]:
X_train['Sex'].unique()

array(['M', 'F'], dtype=object)

In [477]:
X_train['BP'].unique()

array(['LOW', 'NORMAL', 'HIGH'], dtype=object)

In [478]:
X_train['Cholesterol'].unique()

array(['NORMAL', 'HIGH'], dtype=object)

In [479]:
X_train['Na_to_K_binned'].unique()

['>30', '10-20', '20-30', '<10']
Categories (4, object): ['<10' < '10-20' < '20-30' < '>30']

In [480]:
myTest = X_train.iloc[0:2]
myTest

,Age,Sex,BP,Cholesterol,Na_to_K,Age_binned,Na_to_K_binned
131,52,M,LOW,NORMAL,32.922,50s,>30
96,58,F,LOW,HIGH,38.247,50s,>30


In [481]:
encoder=OneHotEncoder()

In [482]:
X_train= pd.DataFrame(encoder.fit_transform(X_train).toarray())
X_test=pd.DataFrame(encoder.fit_transform(X_test).toarray())

In [483]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,210
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


logistic regression

In [484]:
from imblearn.over_sampling import SMOTE
X_train,y_train=SMOTE().fit_resample(X_train,y_train)

In [485]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

encoder=OneHotEncoder(sparse_output=False)
X_encoded=pd.DataFrame(encoder.fit_transform(X),columns=encoder.get_feature_names_out(X.columns))
#split the data into train and test sets
X_train,X_test,y_train,y_test=train_test_split(X_encoded,y,test_size=0.3,random_state=0)

#handle class inbalance with SMOTE
smote=SMOTE(random_state=0)


#Train the Logistic Regression model

model=LogisticRegression()
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
y_pred

# Print classification report and confusion matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Calculate accuracy
# LRAcc = accuracy_score(y_test, y_pred)
# print('Logistic Regression accuracy is: {:.2f}%'.format(LRAcc * 100))
print("accuracy score:",accuracy_score(y_test,y_pred))


              precision    recall  f1-score   support

       drugA       0.67      0.80      0.73         5
       drugB       0.75      1.00      0.86         3
       drugC       0.60      0.75      0.67         4
       drugX       0.81      0.94      0.87        18
       drugY       0.88      0.70      0.78        30

    accuracy                           0.80        60
   macro avg       0.74      0.84      0.78        60
weighted avg       0.81      0.80      0.80        60

[[ 4  0  0  0  1]
 [ 0  3  0  0  0]
 [ 0  0  3  0  1]
 [ 0  0  0 17  1]
 [ 2  1  2  4 21]]
accuracy score: 0.8


knn classifier

In [526]:
# from imblearn.over_sampling import SMOTE
# X_train,y_train=SMOTE().fit_resample(X_train,y_train)
# smote=SMOTE(random_state=0)

In [527]:
# encoder=OneHotEncoder(sparse_output=False)
# X_encoded=pd.DataFrame(encoder.fit_transform(X),columns=encoder.get_feature_names_out(X.columns))


In [528]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=15)

In [529]:
# #split the data into train and test sets
# X_train,X_test,y_train,y_test=train_test_split(X_encoded,y,test_size=0.3,random_state=0)

In [530]:
smote=SMOTE(random_state=0)

In [531]:
y_pred=knn.predict(X_test)
y_pred

array(['drugC', 'drugX', 'drugC', 'drugA', 'drugY', 'drugC', 'drugX',
       'drugC', 'drugC', 'drugX', 'drugA', 'drugY', 'drugY', 'drugX',
       'drugB', 'drugC', 'drugB', 'drugX', 'drugA', 'drugA', 'drugA',
       'drugX', 'drugX', 'drugC', 'drugC', 'drugA', 'drugX', 'drugB',
       'drugA', 'drugB', 'drugX', 'drugC', 'drugA', 'drugX', 'drugC',
       'drugB', 'drugC', 'drugC', 'drugA', 'drugX', 'drugA', 'drugB',
       'drugA', 'drugX', 'drugB', 'drugX', 'drugX', 'drugX', 'drugA',
       'drugB', 'drugC', 'drugB', 'drugA', 'drugB', 'drugC', 'drugC',
       'drugX', 'drugX', 'drugX', 'drugA'], dtype=object)

In [532]:
from sklearn.metrics import confusion_matrix
cmd=confusion_matrix(y_test,y_pred)
cmd

array([[ 4,  1,  0,  0,  0],
       [ 1,  2,  0,  0,  0],
       [ 0,  0,  4,  0,  0],
       [ 1,  1,  4, 12,  0],
       [ 8,  6,  7,  6,  3]])

In [533]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       drugA       0.29      0.80      0.42         5
       drugB       0.20      0.67      0.31         3
       drugC       0.27      1.00      0.42         4
       drugX       0.67      0.67      0.67        18
       drugY       1.00      0.10      0.18        30

    accuracy                           0.42        60
   macro avg       0.48      0.65      0.40        60
weighted avg       0.75      0.42      0.37        60



In [534]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4166666666666667

naive_bayes

In [518]:
from imblearn.over_sampling import SMOTE
X_train,y_train=SMOTE().fit_resample(X_train,y_train)
smote=SMOTE(random_state=0)

In [519]:

encoder=OneHotEncoder(sparse_output=False)
X_encoded=pd.DataFrame(encoder.fit_transform(X),columns=encoder.get_feature_names_out(X.columns))

In [520]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [521]:
from imblearn.over_sampling import SMOTE
X_train,y_train=SMOTE().fit_resample(X_train,y_train)
smote=SMOTE(random_state=0)

In [522]:
y_pred_nb=model.predict(X_test)
y_pred_nb

array(['drugX', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX', 'drugX',
       'drugX', 'drugC', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX',
       'drugB', 'drugC', 'drugY', 'drugY', 'drugX', 'drugA', 'drugA',
       'drugX', 'drugX', 'drugY', 'drugY', 'drugX', 'drugX', 'drugY',
       'drugY', 'drugX', 'drugX', 'drugC', 'drugA', 'drugX', 'drugY',
       'drugY', 'drugX', 'drugC', 'drugA', 'drugX', 'drugY', 'drugA',
       'drugA', 'drugX', 'drugY', 'drugX', 'drugX', 'drugY', 'drugA',
       'drugB', 'drugY', 'drugB', 'drugY', 'drugY', 'drugC', 'drugY',
       'drugX', 'drugX', 'drugY', 'drugB'], dtype='<U5')

In [523]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_nb)
cm

array([[ 4,  0,  0,  0,  1],
       [ 0,  3,  0,  0,  0],
       [ 0,  0,  3,  1,  0],
       [ 0,  0,  0, 17,  1],
       [ 3,  1,  2,  4, 20]])

In [524]:
print(classification_report(y_test,y_pred_nb))

              precision    recall  f1-score   support

       drugA       0.57      0.80      0.67         5
       drugB       0.75      1.00      0.86         3
       drugC       0.60      0.75      0.67         4
       drugX       0.77      0.94      0.85        18
       drugY       0.91      0.67      0.77        30

    accuracy                           0.78        60
   macro avg       0.72      0.83      0.76        60
weighted avg       0.81      0.78      0.78        60



In [525]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_nb)


0.7833333333333333

In [501]:
# from sklearn.naive_bayes import CategoricalNB

# # Train the Categorical Naive Bayes model
# nb_classifier = CategoricalNB()
# nb_classifier.fit(X_train_res, y_train_res)
# y_pred_nb = nb_classifier.predict(X_test)

# # Print classification report and confusion matrix
# print(classification_report(y_test, y_pred_nb))
# print(confusion_matrix(y_test, y_pred_nb))

# # Calculate accuracy
# NBAcc = accuracy_score(y_test, y_pred_nb)
# print('Categorical Naive Bayes accuracy is: {:.2f}%'.format(NBAcc * 100))

In [502]:
# from sklearn.naive_bayes import CategoricalNB
# category = CategoricalNB()

In [503]:
# y_pred_nb=category.predict(X_test)
# y_pred_nb

Decision Tree

In [542]:
from imblearn.over_sampling import SMOTE
X_train,y_train=SMOTE().fit_resample(X_train,y_train)
smote=SMOTE(random_state=0)

In [543]:
encoder=OneHotEncoder(sparse_output=False)
X_encoded=pd.DataFrame(encoder.fit_transform(X),columns=encoder.get_feature_names_out(X.columns))


In [544]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(criterion='entropy')
model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [545]:
y_pred=model.predict(X_test)
y_pred

array(['drugC', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX', 'drugX',
       'drugX', 'drugY', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX',
       'drugB', 'drugY', 'drugY', 'drugY', 'drugX', 'drugA', 'drugA',
       'drugX', 'drugX', 'drugX', 'drugY', 'drugX', 'drugX', 'drugY',
       'drugY', 'drugY', 'drugX', 'drugY', 'drugA', 'drugX', 'drugY',
       'drugY', 'drugX', 'drugY', 'drugA', 'drugX', 'drugY', 'drugA',
       'drugA', 'drugX', 'drugY', 'drugX', 'drugX', 'drugY', 'drugA',
       'drugB', 'drugY', 'drugB', 'drugA', 'drugY', 'drugC', 'drugY',
       'drugX', 'drugX', 'drugY', 'drugB'], dtype=object)

In [546]:
from sklearn.metrics import confusion_matrix
cmd=confusion_matrix(y_test,y_pred)
cmd

array([[ 4,  0,  0,  0,  1],
       [ 0,  3,  0,  0,  0],
       [ 0,  0,  2,  0,  2],
       [ 0,  0,  0, 17,  1],
       [ 4,  1,  0,  4, 21]])

In [547]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       drugA       0.50      0.80      0.62         5
       drugB       0.75      1.00      0.86         3
       drugC       1.00      0.50      0.67         4
       drugX       0.81      0.94      0.87        18
       drugY       0.84      0.70      0.76        30

    accuracy                           0.78        60
   macro avg       0.78      0.79      0.75        60
weighted avg       0.81      0.78      0.78        60



In [548]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7833333333333333

Random Forest

In [563]:
from imblearn.over_sampling import SMOTE
X_train,y_train=SMOTE().fit_resample(X_train,y_train)
smote=SMOTE(random_state=0)

encoder=OneHotEncoder(sparse_output=False)
X_encoded=pd.DataFrame(encoder.fit_transform(X),columns=encoder.get_feature_names_out(X.columns))

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [564]:
y_pred=rf.predict(X_test)
y_pred

array(['drugY', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX', 'drugX',
       'drugX', 'drugC', 'drugX', 'drugY', 'drugY', 'drugY', 'drugX',
       'drugB', 'drugC', 'drugY', 'drugY', 'drugX', 'drugA', 'drugA',
       'drugX', 'drugX', 'drugX', 'drugY', 'drugX', 'drugX', 'drugY',
       'drugY', 'drugY', 'drugX', 'drugC', 'drugA', 'drugX', 'drugY',
       'drugY', 'drugX', 'drugC', 'drugY', 'drugX', 'drugY', 'drugA',
       'drugY', 'drugX', 'drugY', 'drugX', 'drugX', 'drugY', 'drugY',
       'drugB', 'drugY', 'drugB', 'drugY', 'drugY', 'drugY', 'drugY',
       'drugX', 'drugX', 'drugY', 'drugB'], dtype=object)

In [565]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[ 2,  0,  0,  0,  3],
       [ 0,  3,  0,  0,  0],
       [ 0,  0,  2,  0,  2],
       [ 0,  0,  0, 17,  1],
       [ 2,  1,  2,  4, 21]])

In [566]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       drugA       0.50      0.40      0.44         5
       drugB       0.75      1.00      0.86         3
       drugC       0.50      0.50      0.50         4
       drugX       0.81      0.94      0.87        18
       drugY       0.78      0.70      0.74        30

    accuracy                           0.75        60
   macro avg       0.67      0.71      0.68        60
weighted avg       0.74      0.75      0.74        60



In [567]:
print("accuracy score:",accuracy_score(y_test,y_pred))

accuracy score: 0.75
